<div align="right">
Massimo Nocentini<br>
<br>June 9-17, 2016: Parallelogram Polyominoes: shapes generation, <br>isomorphisms, heuristics, pretty printing
<br>May 31, 2016: trying to add <i>randomization</i>, Polyomino's order
<br>May 25, 2016: Polyominoes
<br>May 23, 2016: n-Queens
</div>
<br>
<div align="center">
<b>Abstract</b><br>
This document collect some applications of <i>backtracking</i> techniques.
</div>

In [7]:
%matplotlib inline
%run ../python-libs/bits.py
%run ../python-libs/timing.py
%run ../python-libs/graycodes.py
%run ../python-libs/symbols.py

In [2]:
import sys

sys.setrecursionlimit(10000000)

# $n$-Queens problem

In this section we provide a pythonic implementation of the $n$-Queens problem, using the approach described by [Ruskey][ruskey] in Chapter 3 of his book [Combinatorial Generation][cg]. We use three *bit masks*, namely integers, to represent whether a row, a raising $\nearrow$ and a falling $\searrow$ diagonal are "under attack" by an already placed queen, instead of three boolean arrays. It is sufficient to use *one* bit only to represent that a cell on a diagonal is under attack because, namely *to each diagonal is associated one bit* according to:

   - if such diagonal is raising, call it $d_\nearrow$, then $a_{r_{1}, c_{1}}\in d_\nearrow \wedge a_{r_{2}, c_{2}} \in d_\nearrow$ if and only if $r_{1}+c_{1}=r_{2}+c_{2}$; in words, the sum of the row and column indices is constant along raising diagonals; therefore, diagonal $d_\nearrow$ is associated to the bit in position $r_{1}+c_{1}$ of a suitable bitmask.
   - if such diagonal is falling, call it $d_\searrow$, then $a_{r_{1}, c_{1}}\in d_\searrow \wedge a_{r_{2}, c_{2}} \in d_\searrow$ if and only if $c_{1}-r_{1}=c_{2}-r_{2}$; in words, the difference of the column and row indices is constant along falling diagonals; therefore, diagonal $d_\searrow$ is associated to the bit in position  $c_{1}-r_{1}$, of a suitable bitmask $p$. In order to be consistent, if $c_{1}-r_{1} < 0$ then take the difference modulo $2n-1$, where $n$ is the number of rows (and columns), namely:
   $$
       p_{n-1}\,p_{n-2}\,\ldots\,p_{0}p_{-1}\,p_{-2}\,\ldots\,p_{-(n-1)} \rightarrow 
       p_{-1\mod(2n-1)=2n-2}\,p_{-2\mod(2n-1)=2n-3}\,\ldots\,p_{-(n-1)\mod(2n-1)=n}p_{n-1}\,p_{n-2}\,\ldots\,p_{0}
   $$
   
where rows and cols indexes range in $\lbrace 0,\ldots,n-1 \rbrace$; in both cases, it is necessary a bitmask $2n-1$ bits long. Here's the code:

[ruskey]:http://webhome.cs.uvic.ca/~ruskey/
[cg]:http://www.1stworks.com/ref/RuskeyCombGen.pdf

In [207]:
def queens(n):
    """
    Return an iterable of solutions to the `n`-Queens problem.
    """
    sol = [0] * n
    
    def gen(c, rows, raises, falls):
        
        for r in range(n):
            
            raising = c + r
            falling = (c - r) % (2*n-1) # we use a modular ring in order to handle the case r > c, in this way
                                        # negative positions appear in the most significant part of `falls`
            
            if is_on(rows, r) and is_on(raises, raising) and is_on(falls, falling):
                
                sol[c] = r
                
                yield from [sol] if c == n-1 else gen(c+1, 
                                                      clear_bit(rows, r), 
                                                      clear_bit(raises, raising), 
                                                      clear_bit(falls, falling))
                
    return gen(0, set_all(n), set_all(2*n-1), set_all(2*n-1))

def pretty(sol):
    n = len(sol)
    s = ""
    for r in range(n):
        pos = sol.index(r)
        s += "|{}|\n".format("|".join('Q' if c == pos else ' ' for c in range(n)))
    return s

Simple case with a $5\times 5$ board, so an instance of $5$-Queens problem:

In [208]:
for i, s in enumerate(queens(5)):
    print("sol {}:\n{}".format(i, pretty(s)))

sol 0:
|Q| | | | |
| | | |Q| |
| |Q| | | |
| | | | |Q|
| | |Q| | |

sol 1:
|Q| | | | |
| | |Q| | |
| | | | |Q|
| |Q| | | |
| | | |Q| |

sol 2:
| | |Q| | |
|Q| | | | |
| | | |Q| |
| |Q| | | |
| | | | |Q|

sol 3:
| | | |Q| |
|Q| | | | |
| | |Q| | |
| | | | |Q|
| |Q| | | |

sol 4:
| |Q| | | |
| | | |Q| |
|Q| | | | |
| | |Q| | |
| | | | |Q|

sol 5:
| | | | |Q|
| | |Q| | |
|Q| | | | |
| | | |Q| |
| |Q| | | |

sol 6:
| |Q| | | |
| | | | |Q|
| | |Q| | |
|Q| | | | |
| | | |Q| |

sol 7:
| | | | |Q|
| |Q| | | |
| | | |Q| |
|Q| | | | |
| | |Q| | |

sol 8:
| | | |Q| |
| |Q| | | |
| | | | |Q|
| | |Q| | |
|Q| | | | |

sol 9:
| | |Q| | |
| | | | |Q|
| |Q| | | |
| | | |Q| |
|Q| | | | |



A more demanding case $20$-Queens, build the generator for solutions:

In [212]:
more_queens = queens(20)

and ask for the first 5 of them; in this case we see that some time is required to reach the first leaf in the *backtracking tree* which is also a valid solution.

In [213]:
for i in range(5):
    with timing(lambda: next(more_queens)) as (s, elapsed_time):
        print("Solution computed in {} time:\n{}".format(elapsed_time, pretty(s)))

Solution computed in 0:00:03.204676 time:
|Q| | | | | | | | | | | | | | | | | | | |
| | | |Q| | | | | | | | | | | | | | | | |
| |Q| | | | | | | | | | | | | | | | | | |
| | | | |Q| | | | | | | | | | | | | | | |
| | |Q| | | | | | | | | | | | | | | | | |
| | | | | | | | | | | | | | | | | | |Q| |
| | | | | | | | | | | | | | | | |Q| | | |
| | | | | | | | | | | | | | |Q| | | | | |
| | | | | | | | | | | |Q| | | | | | | | |
| | | | | | | | | | | | | | | |Q| | | | |
| | | | | | | | | | | | | | | | | | | |Q|
| | | | | | | |Q| | | | | | | | | | | | |
| | | | | |Q| | | | | | | | | | | | | | |
| | | | | | | | | | | | | | | | | |Q| | |
| | | | | | |Q| | | | | | | | | | | | | |
| | | | | | | | | | | | |Q| | | | | | | |
| | | | | | | | | | |Q| | | | | | | | | |
| | | | | | | | |Q| | | | | | | | | | | |
| | | | | | | | | | | | | |Q| | | | | | |
| | | | | | | | | |Q| | | | | | | | | | |

Solution computed in 0:00:00.000101 time:
|Q| | | | | | | | | | | | | | | | | | | |
| | | |Q| | | | | | | | | | | | |

# Polyominoes problems

In this section we play with some problems concerning [polyominoes][poly], formalized by prof. [Golomb][golomb] and extended in various directions; we got interest in this topic after reading the chapter about backtracking in the volume of Ruskey.

### Backtracking machanism

Maybe the hardest part is understand how to represent the board and the state (free/occupied) of each cell; moreover, the question about how a shape, and its orientation, is interesting too. We answer to each one:

   - a board with $r$ rows and $c$ columns
     $$
     \begin{array}{c|c|c|c|c}
     0 & r & 2r & \ldots & (c-1)r \\
     \hline
     1 & r+1 & 2r+1 & \ldots & (c-1)r+1 \\
     \hline
     \vdots & \vdots & \vdots & \ddots & \vdots \\
     \hline
     r-1 & 2r-1 & 3r-1 & \ldots & rc-1\\
     \end{array}
     $$
     is represented by an *integer* with $rc$ bits; this is because we want to use *bit masking* techniques and
     it is efficient to find the next *free* cell (using the utility function `low_bit`), which correspond to
     the position of the first bit 1 from the right, namely the least significant part.
   - a *shape* is a collection of cells, usually sharing an edge pairwise. We choose to represent a shape as a
     `lambda` expression, that consumes the *anchor* position as a pair of row and column indices, 
     and returns a list of
     orientations, namely positions coding a possible symmetry, reflection or rotation of the shape; therefore, 
     each orientation is a sequence of positions too. 
     
     By *anchor* we mean the position in which the top-left cell
     of a shape orientation will be placed in the next *free* cell in the board; every orientation should be
     relative to the *anchor* provided, here an example, where the *anchor* is _always_ given at position `(r,c)`
     
                *                     (r-2,c+2)
                *   ->                (r-1,c+2)
            * * *       (r,c) (r,c+1) (r, c+2)
     so the orientation is coded as the iterable `((r,c), (r,c+1), (r-2,c+2), (r-1,c+2), (r, c+2))`; observe how
     pair are listed according to the order *top to bottom* and, when rows are exausted, go the the top of the
     next column and repeat, so *left to right*. The section about *pentominoes* contains many examples 
     of shapes coding.

In order to structure our thoughts, we start with the definition of the *shape* concept as a `namedtuple` object:

[poly]:https://en.wikipedia.org/wiki/Polyomino
[golomb]:https://en.wikipedia.org/wiki/Solomon_W._Golomb

In [1]:
from collections import namedtuple

shape_spec = namedtuple('shape_spec', ['name', 'isomorphisms',])

The following code implements a *backtracking* function that yields a sequence of tiling of a given board, possibly containing some forbidden cells, according to a set of available shapes.

In [41]:
import math, random

def polyominoes(dim, shapes, availables='ones', max_depth_reached=None, 
                forbidden=[], simulated_annealing={'enabled':False, 'energy':lambda s: 0, 'scheduler':None}, 
                nogood=lambda shape_placement, fewer_positions, iso, availables: False):
    """
    Returns an iterable of arrangements for the Polyominos problem.
    """
    rows, cols = dim
    sol = []
    if not availables or availables == 'ones':
        availables = {s.name:1 for s in shapes} #[1]*len(shapes) 
    elif availables == 'inf': 
        availables = {s.name:-1 for s in shapes} #[-1]*len(shapes) # trick: represent ∞ piece availability by decreasing negative numbers
    
    def place(S, positions):
        for r, c in positions:
            S = clear_bit(S, r + rows*c)
        return S
    
    def shuffle_shapes():
        shuffling = list(shapes)
        random.shuffle(shuffling)
        return shuffling
    
    initial_temperature = sum(availables[s.name] for s in shapes)
    schedulers = {'cauchy': lambda t: initial_temperature/(t),
                  'standard':lambda t: initial_temperature*.95**t,
                  'log':lambda t: initial_temperature/math.log(t),}
    
    def simulated_annealing_choice(step, competitors, temperature_scheduler, boltzman_const=1):
    
        candidate, outsider = competitors
        energy = simulated_annealing['energy']
        delta = energy(outsider) - energy(candidate) # in this way it is always positive
        
        if delta > 0: return outsider
        elif not delta: return candidate if random.random() > 0.5 else outsider
        
        # therefore: delta < 0
        prob = math.exp(delta/(boltzman_const * temperature_scheduler(step)))
        return outsider if random.random() < prob else candidate
    
        """
        freq = int(prob*accuracy)
        population = ([o]*freq) + ([s]*(accuracy-freq))
        return random.choice(population)
        """
    
    def gen(positions, attempts):
    
        p = low_bit(positions)
        
        c, r = divmod(p, rows)
        
        for i, s in enumerate(shapes):
            
            if not availables[s.name]: continue
                
            #________________________________________________________________
            
            if simulated_annealing['enabled']:
                left_shapes = {s for s in shapes if availables[s.name]} - {s}
                annealing_shape = left_shapes.pop() if left_shapes else s

                s = simulated_annealing_choice(step=sum(availables[s.name] for s in shapes), 
                                               competitors=(s, annealing_shape),
                                               temperature_scheduler=schedulers[simulated_annealing['scheduler']])
                
            #________________________________________________________________
            
            for j, iso in enumerate(s.isomorphisms(r, c)):

                if all(0 <= rr < rows and 0 <= cc < cols and is_on(positions, rr + rows*cc) for rr, cc in iso):

                    fewer_positions = place(positions, iso)
                    
                    if nogood((s,p), fewer_positions, iso, availables): continue

                    availables[s.name] -= 1
                    sol.append((s, positions, (r,c), iso),)

                    yield from [sol] if not (fewer_positions and attempts) else gen(fewer_positions, attempts-1)

                    sol.pop()
                    availables[s.name] += 1
        
    return gen(place(set_all(rows*cols), forbidden), max_depth_reached or -1)


### Pretty printers

The following pretty printer uses different symbols (mainly fetched from Greek and Latin alphabets) for different shapes:

In [63]:
def symbols_pretty(sol, dim, symbols, missing_symbol=' ', joiner=lambda board: '\n'.join(board), axis=False):
    
    from collections import defaultdict
    
    table = defaultdict(lambda: missing_symbol)
    for s, _, _, iso in sol:
        table.update({(r, c): symbols[s.name] for r, c in iso})
    
    rows, cols = dim    
    board = []
    
    board.append("┌" + ("─" * (2*cols + 1)) + (">" if axis else "┐"))
    for r in range(rows): 
        board.append('│ ' + ' '.join(table[r, c] for c in range(cols)) + ('  ' if axis else ' │'))
    board.append("v" + (" " * 2*(cols+1))  if axis else "└" + ("─" * (2*cols + 1)) + "┘")
    
    return joiner(board)

On the other hand, the following pretty printer has a more *schematic* spirit because it draws tilings using vertical bars and underscores (`|` and `_`):

In [64]:
def tiling_pretty(sol, dim, symbols, missing_symbol='●', joiner=lambda board: '\n'.join(board)):
    
    from string import whitespace
    
    table = {}
    for s, _, _, iso in sol:
        table.update({(r,c):s.name for r, c in iso})
    
    rows, cols = dim
    
    matrix = []
    matrix.append(' ' + ' '.join(['_' if (0, c) in table else ' ' for c in range(cols)]))
    
    for r in range(rows): 
        row = '|'
        for c in range(cols):
            
            if (r, c) not in table:
                row += missing_symbol + ('|' if (r, c+1) in table else ' ')
            else:
                if (r+1, c) in table:
                    row += '_' if table[r, c] != table[r+1, c] else ' '
                else:
                    row += '_'
            
                if (r, c+1) in table:
                    row += '|' if table[r, c] != table[r, c+1] else ' '
                else:
                    row += '|' #' '
                    
                
                #row += '_' if (r+1, c) in table and table[r, c] != table[r+1, c] else ' '
                #row += '|' if (r, c+1) in table and table[r, c] != table[r, c+1] else ' '
                
                
        matrix.append(row)
    
    return joiner(matrix)

At last, the following is a *decorator* pretty printer, since it comsumes an iterable of tilings and applies a given pretty printer in order to build a `Markdown` object for a nicer rendering:

In [65]:
def markdown_pretty(tilings, dim, shapes, raw_text=False, pretty_printer=tiling_pretty,
                    symbols=(lower_greek_symbols() + capital_greek_symbols() + latin_symbols())):
    """
    I am a generator of tilings, pretty-printed as Markdown objects.
    """

    from IPython.display import Markdown
    import itertools

    def row_tabulator(row): return ('' if raw_text else '\t') + row # '\t' necessary for nice Markdown rendering
    
    for i in itertools.count():    
        
        # since I'm a generator on top of `tiling`, which is a generator too, 
        # if it raises `StopIteration`, I propagate that as well.
        with timing(lambda: next(tilings)) as (s, elapsed_time): 
        
            prettier = pretty_printer(s, dim, {s.name:symbols[i] for i,s in enumerate(shapes)}, 
                                      joiner=lambda board: '\n'.join(map(row_tabulator, board)))

            code = "Solution $s_{{{}}}$ computed in __{}__ time, respect last solution elapsed time (which is 0 if the current is the very first one):\n{}".format(i, elapsed_time, prettier)

            yield code if raw_text else Markdown(code)

## Pentominoes

The first instance we would like to study are shapes composed of 5 cells, also called [pentominoes][penta]; the next code define all shapes and their orientations: 

[penta]:https://en.wikipedia.org/wiki/Pentomino

In [66]:
"""
  *
* * *
  *
"""
X_shape = shape_spec(name='X', 
                     isomorphisms=lambda r, c: [((r, c), (r-1, c+1), (r, c+1), (r+1, c+1), (r, c+2))])

"""
*
*
*
*
*

* * * * *
"""
I_shape = shape_spec(name='I',
                     isomorphisms=lambda r, c: [((r, c), (r+1,c), (r+2,c), (r+3,c), (r+4,c)),
                                                ((r, c), (r,c+1), (r,c+2), (r,c+3), (r,c+4))])

"""
*
*
* * *

* * *
    *
    *

    *
    *
* * *

* * *
*
*
"""
V_shape = shape_spec(name='V', 
                     isomorphisms=lambda r, c: [((r,c), (r+1,c), (r+2,c), (r+2, c+1), (r+2, c+2)),
                                                ((r,c), (r, c+1), (r,c+2), (r+1, c+2), (r+2, c+2)),
                                                ((r,c), (r,c+1), (r-2,c+2), (r-1,c+2), (r, c+2)),
                                                ((r,c), (r+1,c), (r+2,c), (r,c+1), (r, c+2))])

"""
* *
*
* *

* * *
*   *

*   *
* * *

* *
  *
* *  
"""
U_shape = shape_spec(name='U',
                     isomorphisms=lambda r, c: [((r,c), (r+1,c), (r+2,c), (r,c+1), (r+2,c+1)),
                                                ((r,c), (r+1,c), (r, c+1), (r,c+2), (r+1,c+2)),
                                                ((r,c), (r+1, c), (r+1, c+1), (r, c+2), (r+1, c+2)),
                                                ((r,c), (r+2, c), (r, c+1), (r+1, c+1), (r+2, c+1)),])

"""
*
* *
  * *
  
* *
  * *
    *

    *
  * *
* *

  * *
* *
*
"""
W_shape = shape_spec(name='W',
                     isomorphisms=lambda r, c: [((r,c), (r+1,c), (r+1,c+1), (r+2,c+1), (r+2,c+2)),
                                                ((r,c), (r,c+1), (r+1,c+1), (r+1,c+2), (r+2,c+2)),
                                                ((r,c), (r-1,c+1), (r,c+1), (r-2,c+2), (r-1,c+2)),
                                                ((r,c), (r+1,c), (r-1,c+1), (r,c+1), (r-1,c+2)),])

"""
* * *
  *
  *
  
*
* * *
*

    *
* * *
    *

  *
  *
* * *
"""
T_shape = shape_spec(name='T',
                     isomorphisms=lambda r, c: [((r,c), (r,c+1), (r+1,c+1), (r+2,c+1), (r,c+2)),
                                                ((r,c), (r+1,c), (r+2,c), (r+1,c+1), (r+1,c+2)),
                                                ((r,c), (r,c+1), (r-1,c+2), (r,c+2), (r+1,c+2)),
                                                ((r,c), (r-2,c+1), (r-1,c+1), (r,c+1), (r,c+2)),])

"""
*
* * *
    *

    *
* * *
*

  * *  
  * 
* *

* *
  *
  * *
"""
Z_shape = shape_spec(name='Z', 
                     isomorphisms=lambda r, c: [((r,c), (r+1,c), (r+1,c+1), (r+1,c+2), (r+2,c+2)),
                                                ((r,c), (r+1,c), (r,c+1), (r-1,c+2), (r,c+2)),
                                                ((r,c), (r-2,c+1), (r-1,c+1), (r,c+1), (r-2,c+2)),
                                                ((r,c), (r,c+1), (r+1,c+1), (r+2,c+1), (r+2,c+2)),])

"""
*
* *
  *
  *
  
  *
* *
*
*

*
*
* *
  *
  
  *
  *
* *
*

  * * *
* *

* *
  * * *

* * *
    * *

    * *
* * *
"""
N_shape = shape_spec(name='N',
                     isomorphisms=lambda r, c: [((r,c), (r+1,c), (r+1,c+1), (r+2,c+1), (r+3,c+1)),
                                                ((r,c), (r+1,c), (r+2,c), (r-1,c+1), (r,c+1)),
                                                ((r,c), (r+1,c), (r+2,c), (r+2,c+1), (r+3,c+1)),
                                                ((r,c), (r+1,c), (r-2,c+1), (r-1,c+1), (r,c+1)),
                                                ((r,c), (r-1,c+1), (r,c+1), (r-1,c+2), (r-1,c+3)),
                                                ((r,c), (r,c+1), (r+1,c+1), (r+1,c+2), (r+1,c+3)),
                                                ((r,c), (r,c+1), (r,c+2), (r+1,c+2), (r+1,c+3)),
                                                ((r,c), (r,c+1), (r-1,c+2), (r,c+2), (r-1,c+3)),])

"""
*
*
*
* *
  
  *
  *
  *
* *

* *
  *
  *
  *
  
* *
*
*
*

*
* * * *

* * * *
*

      *
* * * *

* * * *
      *

"""
L_shape = shape_spec(name='L',
                     isomorphisms=lambda r, c: [((r,c), (r+1,c), (r+2,c), (r+3,c), (r+3,c+1)),
                                                ((r,c), (r-3,c+1), (r-2,c+1), (r-1,c+1), (r,c+1)),                        
                                                ((r,c), (r,c+1), (r+1,c+1), (r+2,c+1), (r+3,c+1)),                        
                                                ((r,c), (r+1,c), (r+2,c), (r+3,c), (r,c+1)), 
                                                ((r,c), (r+1,c), (r+1,c+1), (r+1,c+2), (r+1,c+3)),                        
                                                ((r,c), (r+1,c), (r,c+1), (r,c+2), (r,c+3)),                        
                                                ((r,c), (r,c+1), (r,c+2), (r-1,c+3), (r,c+3)),                        
                                                ((r,c), (r,c+1), (r,c+2), (r,c+3), (r+1,c+3)),])

"""
*
* *
*
*

*
*
* *
*

  *
* *
  *
  *
  
  *
  *
* *
  *

* * * *
    *
    
    *
* * * *

* * * *
  *
  
  *
* * * *    
"""
Y_shape = shape_spec(name='Y', 
                     isomorphisms=lambda r, c: [((r,c), (r+1,c), (r+2,c), (r+3,c), (r+1,c+1)),
                                                ((r,c), (r+1,c), (r+2,c), (r+3,c), (r+2,c+1)),                        
                                                ((r,c), (r-1,c+1), (r,c+1), (r+1,c+1), (r+2,c+1)),                        
                                                ((r,c), (r-2,c+1), (r-1,c+1), (r,c+1), (r+1,c+1)),                        
                                                ((r,c), (r,c+1), (r,c+2), (r+1,c+2), (r,c+3)),                        
                                                ((r,c), (r,c+1), (r-1,c+2), (r,c+2), (r,c+3)),                        
                                                ((r,c), (r,c+1), (r+1,c+1), (r,c+2), (r,c+3)),                        
                                                ((r,c), (r-1,c+1), (r,c+1), (r,c+2), (r,c+3)),])

"""
*
* * *
  *
  
    *
* * *
  *
 
  *
* * *
*

  *
* * *
    *
  
  *
  * *
* *

  *
* *
  * *
  
* *
  * *
  *
  
  * *
* *
  *
"""
F_shape = shape_spec(name='F',
                     isomorphisms=lambda r, c: [((r,c), (r+1,c), (r+1,c+1), (r+2,c+1), (r+1,c+2)),
                                                ((r,c), (r,c+1), (r+1,c+1), (r-1,c+2), (r,c+2)),                        
                                                ((r,c), (r+1,c), (r-1,c+1), (r,c+1), (r,c+2)),                        
                                                ((r,c), (r-1,c+1), (r,c+1), (r,c+2), (r+1,c+2)),                                                
                                                ((r,c), (r-2,c+1), (r-1,c+1), (r,c+1), (r-1,c+2)),                                                
                                                ((r,c), (r-1,c+1), (r,c+1), (r+1,c+1), (r+1,c+2)),                                                
                                                ((r,c), (r,c+1), (r+1,c+1), (r+2,c+1), (r+1,c+2)),
                                                ((r,c), (r-1,c+1), (r,c+1), (r+1,c+1), (r-1,c+2)),])

"""
*
* *
* *

  *
* *
* *

  * *
* * *

* *
* * *

* * *
* *

* * *
  * *
  
* *
* *
*

* *
* *
  *
"""
P_shape = shape_spec(name='P', 
                     isomorphisms=lambda r, c: [((r,c), (r+1,c), (r+2,c), (r+1,c+1), (r+2,c+1)),
                                                ((r,c), (r+1,c), (r-1,c+1), (r,c+1), (r+1,c+1)),                        
                                                ((r,c), (r-1,c+1), (r,c+1), (r-1,c+2), (r,c+2)),                                                
                                                ((r,c), (r+1,c), (r,c+1), (r+1,c+1), (r+1,c+2)),                                                
                                                ((r,c), (r+1,c), (r,c+1), (r+1,c+1), (r,c+2)),                                                  
                                                ((r,c), (r,c+1), (r+1,c+1), (r,c+2), (r+1,c+2)),                                                
                                                ((r,c), (r+1,c), (r+2,c), (r,c+1), (r+1,c+1)),                        
                                                ((r,c), (r+1,c), (r,c+1), (r+1,c+1), (r+2,c+1)),])


In order to tile a board, we use all of them:

In [67]:
shapes =  [X_shape, I_shape, V_shape, U_shape, W_shape, T_shape, 
           Z_shape, N_shape, L_shape, Y_shape, F_shape, P_shape]

### Using *one* piece for each shape

so, we are now ready to tile a board with 6 rows and 10 columns, where no cell is forbidden and only *one* piece is available for each shape: 

In [68]:
dim = (6,10)

polys_sols = polyominoes(dim, shapes, availables="ones", forbidden=[])

In [79]:
tilings = markdown_pretty(polys_sols, dim, shapes, pretty_printer=symbols_pretty, raw_text=True)

and pretty print the first 10 solutions:

In [80]:
for i in range(10):
    print(next(tilings))

Solution $s_{0}$ computed in __0:00:00.447627__ time, respect last solution elapsed time (which is 0 if the current is the very first one):
┌─────────────────────┐
│ β ε ε μ μ μ ζ δ δ δ │
│ β κ ε ε μ μ ζ δ θ δ │
│ β κ κ ε α ζ ζ ζ θ θ │
│ β κ γ α α α λ η η θ │
│ β κ γ ι α λ λ λ η θ │
│ γ γ γ ι ι ι ι λ η η │
└─────────────────────┘
Solution $s_{1}$ computed in __0:00:00.108994__ time, respect last solution elapsed time (which is 0 if the current is the very first one):
┌─────────────────────┐
│ β θ θ θ ε ε η η μ μ │
│ β ι ι θ θ ε ε η μ μ │
│ β ι ζ ζ ζ λ ε η η μ │
│ β ι γ ζ λ λ λ α δ δ │
│ β ι γ ζ λ κ α α α δ │
│ γ γ γ κ κ κ κ α δ δ │
└─────────────────────┘
Solution $s_{2}$ computed in __0:00:00.228469__ time, respect last solution elapsed time (which is 0 if the current is the very first one):
┌─────────────────────┐
│ β ι ι ι ι η κ κ κ κ │
│ β ι θ η η η λ λ κ ζ │
│ β θ θ η α λ λ ζ ζ ζ │
│ β θ γ α α α λ ε ε ζ │
│ β θ γ δ α δ ε ε μ μ │
│ γ γ γ δ δ δ ε μ μ μ │
└─────────────────────┘
Solu

Moreover, it is possible to count all solutions, namely tilings, with the following code:

In [ ]:
with timing(lambda: len(list(polys_sols))) as (t, elapsed_time):
    print("{} sols in {} time".format(t, elapsed_time))

### With *forbidden* cells and (not so) many pieces for each shape

On the other hand, the following is another board with some forbidden cells and 3 pieces for each shape are available:

In [83]:
dim = (6,10)
polys_sols = polyominoes(dim, shapes, availables={s.name:3 for s in shapes},
                         forbidden=[(0,0), (1,0), (2,0), (3,0), (4,0), 
                                    (1,9),(2,9),(3,9),(4,9), (5,9),
                                    (1,5), (2, 4), (2, 5), (3, 4), (3,5)])
tilings = markdown_pretty(polys_sols, dim, shapes, pretty_printer=symbols_pretty, raw_text=True)

and pretty print the first 10 solutions:

In [84]:
for i in range(10):
    print(next(tilings))

Solution $s_{0}$ computed in __0:00:00.011117__ time, respect last solution elapsed time (which is 0 if the current is the very first one):
┌─────────────────────┐
│   β β ι ι ι ι μ μ μ │
│   β β ι ι   ι μ μ   │
│   β β ι     γ ι ι   │
│   β β ι     γ γ ι   │
│   β β ι γ γ γ γ ι   │
│ β β β β β γ γ γ ι   │
└─────────────────────┘
Solution $s_{1}$ computed in __0:00:00.009081__ time, respect last solution elapsed time (which is 0 if the current is the very first one):
┌─────────────────────┐
│   β β ι ι ι ι γ γ γ │
│   β β ι ι   ι γ θ   │
│   β β ι     γ γ θ   │
│   β β ι     γ θ θ   │
│   β β ι γ γ γ θ ι   │
│ β β β β β ι ι ι ι   │
└─────────────────────┘
Solution $s_{2}$ computed in __0:00:00.001385__ time, respect last solution elapsed time (which is 0 if the current is the very first one):
┌─────────────────────┐
│   β β ι ι ι ι ζ ζ ζ │
│   β β ι ι   ι κ ζ   │
│   β β ι     γ κ ζ   │
│   β β ι     γ κ κ   │
│   β β ι γ γ γ κ ι   │
│ β β β β β ι ι ι ι   │
└─────────────────────┘
Solu

Moreover, I've found a board configuration, where cells located at $(1,1)$ and $(4,8)$ are forbidden, such that it takes very long to be tiled, eventually I stopped the search each time, even if *infinitely many* pieces for each shape are available. I believe that even if we use *randomization* at shape/orientation selection time wouldn't help to shrink the complexity. 

### Polyomino's order

Again, in the exercise **7 in Chapter 3**, Ruskey asks to find the *order* of some polyomino. Here's the definition:

>Define the order of a polyomino $P$ to be the smallest number of copies of $P$ that
will fit perfectly into a rectangle, where rotations and reflections of $P$ are allowed.

We reproduce the tiling using the `Y` polyomino and we check that its order is actually 10; in order to show the tiling we given 10 copies of the `Y` shape, each one with one piece available:

In [95]:
dim = (5,10)
repeated_Y_shapes = [shape_spec(name="{}_{}".format(Y_shape.name, i),
                                isomorphisms=Y_shape.isomorphisms) for i in range(10)]
polys_sols = polyominoes(dim, repeated_Y_shapes, availables='ones', forbidden=[])
tilings = markdown_pretty(polys_sols, dim, repeated_Y_shapes, pretty_printer=symbols_pretty, raw_text=True)

there are many way to tile the board, here there are the first three:

In [96]:
for i in range(3):
    print(next(tilings))

Solution $s_{0}$ computed in __0:00:00.089210__ time, respect last solution elapsed time (which is 0 if the current is the very first one):
┌─────────────────────┐
│ α γ γ γ γ ζ ι ι ι ι │
│ α α δ γ ζ ζ ζ ζ ι κ │
│ α δ δ δ δ η η η η κ │
│ α β ε ε ε ε θ η κ κ │
│ β β β β ε θ θ θ θ κ │
└─────────────────────┘
Solution $s_{1}$ computed in __0:00:00.000147__ time, respect last solution elapsed time (which is 0 if the current is the very first one):
┌─────────────────────┐
│ α γ γ γ γ ζ κ κ κ κ │
│ α α δ γ ζ ζ ζ ζ κ ι │
│ α δ δ δ δ η η η η ι │
│ α β ε ε ε ε θ η ι ι │
│ β β β β ε θ θ θ θ ι │
└─────────────────────┘
Solution $s_{2}$ computed in __0:00:00.000179__ time, respect last solution elapsed time (which is 0 if the current is the very first one):
┌─────────────────────┐
│ α γ γ γ γ ζ θ θ θ θ │
│ α α δ γ ζ ζ ζ ζ θ κ │
│ α δ δ δ δ η η η η κ │
│ α β ε ε ε ε ι η κ κ │
│ β β β β ε ι ι ι ι κ │
└─────────────────────┘


## Fibonacci tilings

It is interesting to way we define shape because allows us to build shapes composed of a *different* number of cells. In the following cell we define two shapes that resembles a *Fibonacci* schema:

In [97]:
"""
*
"""
square_shape = shape_spec(name='square',
                          isomorphisms=lambda r, c: [((r, c),)])

"""
* *

*
*
"""
domino_shape = shape_spec(name='domino',
                          isomorphisms=lambda r, c: [((r, c), (r, c+1)),
                                                     ((r, c), (r+1, c))])

collect them together:

In [98]:
fibonacci_shapes = [square_shape, domino_shape]

and use an infinite copy of pieces for each shape to tile a strip with 12 columns:

In [99]:
dim = (1,12)
fibonacci_sols = polyominoes(dim, fibonacci_shapes, availables='inf', forbidden=[])

as the theory confirms, there are 233 way to tile such strip:

In [100]:
with timing(lambda: len(list(fibonacci_sols))) as (t, elapsed_time):
    print("{} sols in {} time".format(t, elapsed_time))

233 sols in 0:00:00.014320 time


To see the pattern, we can tile a greater board:

In [184]:
dim = (3,50)
fibonacci_sols = polyominoes(dim, fibonacci_shapes, availables='inf', forbidden=[])
tilings = markdown_pretty(fibonacci_sols, dim, fibonacci_shapes, pretty_printer=tiling_pretty, raw_text=False)

and pretty print the first 10 solutions (not really interesting, though! many $\alpha$ symbols...maybe we could *randomize* the generator...)

In [185]:
for i in range(10000):
    next(tilings)

In [186]:
next(tilings)

Solution $s_{10000}$ computed in __0:00:00.000129__ time, respect last solution elapsed time (which is 0 if the current is the very first one):
	 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
	|                                                                                        _ _|   |_ _|
	|                                                                                       |           |
	|_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _|_ _ _ _ _ _|

# Parallelogram Polyominoes

In [187]:
def parallelogram_polyominoes(sp):
    """
    Returns the set of shapes denoting all *parallelogram polyominoes* of semiperimeter `sp`.
    
    The relation to the main problem is $sp = n + 2$, where the board edge is $2^{n}$.
    """
    
    import itertools
    
    steps = [(1,0),  # go to cell below
             (0,1),] # go to cell on the right
    
    # we assume the canonical ordering inside a board, namely ascending
    # order from top to bottom, and from left to right, as in cells above.
    
    # first of all build Catalan paths that have no intersection point,
    # using only vertical and horizontal steps (here we do not distinguish
    # among upward/downward vertical steps, what is important is to remain
    # consistent to board ordering).
    
    initial_prefix = [(0,0)] # always start placing cell at anchor coordinate (r, c)
    prefixes = {tuple(initial_prefix)}

    candidates = set()
    
    n = sp - 2
    
    while prefixes:
        
        prefix = prefixes.pop()
        
        if len(prefix) > n: 
            candidates.add(prefix)
            continue
        
        last_row_offset, last_col_offset = prefix[-1]
        dominating_prefixes = [(last_row_offset + ro, last_col_offset + co) for ro, co in steps]
        dominated_prefixes = [(last_row_offset + ro, last_col_offset + co) for ro, co in steps]
        for sub, dom in [(sub,dom) for sub, dom in zip(dominated_prefixes, dominating_prefixes) if sub <= dom]:
            prefixes.add(prefix + (sub,))
            prefixes.add(prefix + (dom,))
    
    parallelograms = {(tuple(dom), tuple(sub)) for sub, dom in itertools.product(candidates, repeat=2) 
                      if sub[-1] == dom[-1]}
    
    polyominoes = {frozenset(fst) | frozenset(snd) for fst, snd in parallelograms}
    
    def fill(polyomino):
        r_max, c_max = max(r for r,_ in polyomino), max(c for _,c in polyomino)
        filled = set()
        for coord in itertools.product(range(1, r_max), range(1, c_max)):
            
            if coord in polyomino: continue
            
            row = [(rr, cc) for rr, cc in polyomino if coord[0] == rr]
            column = [(rr, cc) for rr, cc in polyomino if coord[1] == cc]
        
            if (min(column, default=coord) < coord < max(column, default=coord) and
                min(row, default=coord) < coord < max(row, default=coord)):
        
                filled.add(coord)
                #print("filled: ", polyomino, polyomino | filled)
                
        return polyomino | filled
    
    return {fill(p) for p in polyominoes}
        

### *Isomorphisms*: mirrors and rotations

In [188]:

def rotate_clockwise(shape):
    """
    Returns a shape rotated clockwise by π/2 radians.
    """
    clockwise = [(c, -r) for r, c in shape] # by matrix multiplication for rotations
    #print(clockwise)
    co, ro = min((c, r) for r, c in clockwise)
    #print(ro, co)
    return frozenset((r-ro, c-co) for r, c in clockwise)

def rotate_clockwise_isomorphisms(isos):
    
    rotations = set()
    
    for i in isos:
        rotating = i
        
        rotating = rotate_clockwise(rotating)
        rotations.add(rotating)
        
        rotating = rotate_clockwise(rotating)
        rotations.add(rotating)
        
        rotating = rotate_clockwise(rotating)
        rotations.add(rotating)
        
    #print(isos | frozenset(rotations))
    return isos | rotations

#______________________________________________________________________________

def vertical_mirror(shape):
    m = max([c for r, c in shape])
    return frozenset((r, m-c) for r, c in shape)

def vertical_isomorphism(isos):
    return isos | frozenset(vertical_mirror(i) for i in isos)

#______________________________________________________________________________

def make_shapes(primitive_polyos, isomorphisms=lambda isos: isos):
    
    prefix = "polyo"
    
    def o(j, isos):
        return shape_spec(name='{}{}'.format(prefix, j),
                          isomorphisms=lambda r, c: [ [(r+ro, c+co) for ro, co in iso] for iso in isos])
    
    return [o(j, isomorphisms({p})) for j, p in enumerate(primitive_polyos)]


Sandbox to test:

In [189]:
c = frozenset({(0,0), (1,0), (1,1), (1,2), (1,3)})
print(c)
c = rotate_clockwise(c)
print(c)
c = rotate_clockwise(c)
print(c)
c = rotate_clockwise(c)
print(c)

frozenset({(1, 2), (1, 0), (1, 3), (1, 1), (0, 0)})
frozenset({(3, 0), (2, 0), (1, 0), (0, 0), (0, 1)})
frozenset({(0, 1), (0, 3), (0, 0), (0, 2), (1, 3)})
frozenset({(0, 1), (-3, 1), (-2, 1), (0, 0), (-1, 1)})


## A tiling game from Del Longo, Pinzani et al.

In [190]:
def describe(semiperimeter, polyominoes):
    
    from IPython.display import Markdown    
    
    n, catalan = semiperimeter - 2, semiperimeter - 1
    size=2**n

    area = 0
    for p in polyominoes:
        area += len(p)
    
    theoretical_area = 4**n
    
    code='''
**Problem instance**: 
   - using {pp} parallelogram polyominoes, which is the catalan number $c_{catalan}$ (0-based indexing)
   - each polyomino has semiperimeter $sp$ equals to {sp}, so let $n=sp-2={n}$
   - board edges have size $2^{n}={size}$ each
   - theoretically, area is known to be $4^{n}={fa}$, which *is {pred}* equal to counted area {ca}'''.format(
        sp=semiperimeter, n=n, size=size, pp=len(polyominoes), fa=theoretical_area, 
        pred='not' if theoretical_area != area else '', ca=area, catalan=catalan)
    
    return Markdown(code), n, catalan, polyominoes, theoretical_area, area, size


Using function `describe` above, we state the problem instance clearly with:

In [191]:
semiperimeter = 6
description, *facts = describe(semiperimeter, parallelogram_polyominoes(semiperimeter))
n, catalan, polyos, theoretical_area, area, size = facts

description


**Problem instance**: 
   - using 42 parallelogram polyominoes, which is the catalan number $c_5$ (0-based indexing)
   - each polyomino has semiperimeter $sp$ equals to 6, so let $n=sp-2=4$
   - board edges have size $2^4=16$ each
   - theoretically, area is known to be $4^4=256$, which *is * equal to counted area 256

The **order** in which *shapes* are provided is very important since it affects the quality of backtracking:

In [192]:
#shapes = make_shapes(pp, isomorphisms=vertical_isomorphism)#, isomorphisms=lambda isos: vertical_isomorphism(rotate_clockwise_isomorphisms(isos)))
shapes = make_shapes(polyos)#, isomorphisms=lambda isos: vertical_isomorphism(rotate_clockwise_isomorphisms(isos)))

def convex_hull(iso):
    lower_row, lower_col = min(iso)
    greatest_row, greatest_col = max(iso) # find the "convex hull" respect to the cell at the bottom-right most location
    square = max(greatest_row, greatest_col) + 1
    return [(r, c)
            for r in range(lower_row, square) 
            for c in range(lower_col, square)] # convex hull area

max_convex_hull = max(map(lambda iso: len(convex_hull(iso)), 
                          [iso for s in shapes for iso in s.isomorphisms(0,0)]))

def area_key(s, weights={0:0, 1:0, 2:1}):
    import random
    isos = s.isomorphisms(0,0) # place the shape somewhere, the origin is pretty good
    iso = isos.pop() # all isomorphisms, of course, have the same are, therefore pick the first one
    filled_area = len(iso) # since a piece is represent as an iterable of coordinates `(r, c)`
    convex_hull_area = len(convex_hull(iso))
    #key = filled_area*(weights['filled'] + weights['convex']/convex_hull_area) + weights['bias']*random.random() # return the ratio of the filled area respect the convex hull
    
    #key = 0*max(iso)[1] + filled_area * convex_hull_area / max_convex_hull
    
    def key(x):
        max_r, max_c = max(iso)
        #return max_r + max_c*x + (filled_area * convex_hull_area / max_convex_hull)*x**2
        return (weights[0]*max_r + 
                weights[1]*max_c*x + 
                weights[2]*(filled_area * convex_hull_area / max_convex_hull)*x**2)
    
    return key(2)


shapes = sorted(shapes, key=area_key, reverse=True)

Represent all available parallelogram polyominoes:

In [194]:
polyominoes_boards = [symbols_pretty([(shape, None, (0,0), iso)], 
                                     (semiperimeter, semiperimeter), 
                                     {shape.name:'▢'}, 
                                     joiner=lambda board: board,
                                     axis=True) 
                      for i, shape in enumerate(shapes) for iso in shape.isomorphisms(0,0)]

group=5
for i in range(0, len(polyominoes_boards), group):
    for k in range(semiperimeter+2):
        grouped_board = []
        for j in range(i, i + group):
            if j >= len(polyominoes_boards): break
            grouped_board.append(polyominoes_boards[j][k])
        print(' '.join(grouped_board))



┌─────────────> ┌─────────────> ┌─────────────> ┌─────────────> ┌─────────────>
│ ▢ ▢           │ ▢ ▢ ▢ ▢       │ ▢             │ ▢ ▢ ▢ ▢ ▢     │ ▢            
│ ▢ ▢           │ ▢ ▢ ▢ ▢       │ ▢             │               │ ▢ ▢          
│ ▢ ▢           │               │ ▢             │               │ ▢ ▢          
│ ▢ ▢           │               │ ▢             │               │ ▢ ▢          
│               │               │ ▢             │               │              
│               │               │               │               │              
v               v               v               v               v              
┌─────────────> ┌─────────────> ┌─────────────> ┌─────────────> ┌─────────────>
│ ▢ ▢           │ ▢ ▢ ▢ ▢       │ ▢ ▢ ▢         │ ▢             │ ▢ ▢ ▢        
│ ▢ ▢           │   ▢ ▢ ▢       │ ▢ ▢ ▢ ▢       │ ▢ ▢           │   ▢ ▢ ▢      
│ ▢ ▢           │               │               │ ▢ ▢           │              
│   ▢           │               │       

Thoughts:
   - proceed by *iterative deepening*
   - at $C_{n}-1$ level it seems that a *hole* (one empty cell surrounded by filled ones) occurs, where $C_{n}$ is 
   the relative Catalan number to the problem instance under study
   - possible pruning if a *hole* is found after piece placement
   - add a keyword argument to require the check about the discover of a solution: if a solution is found ensure
   that a predicate on the `available` array holds (for instance, that every piece has been used so that no pieces 
   left for each solution)

In [204]:
###############  DIRTY  ######################################

dim = (size, size)

def nogood_predicate(placed_shape, fewer_positions, iso, fringe, availables):
    
    if not fewer_positions:
        return False
    
    shape, p = placed_shape
    
    #print("given fringe ", fringe)
    
    rows=size
    c,r = divmod(p, rows)
    
    #if r == rows-1 or c == size-1: return True
    
    def cell_fringe(cell):
        r, c = cell
        return {(r, c-1)}#, (r+1, c)}
        #return {(r, c-1),(r-1, c),(r+1, c),(r, c+1),} # this is the complete fringe
    
    max_iso_cell_r, max_iso_cell_c = max(iso)
    
    
    fringe_iso = {(rr, cc) 
                  for cell in iso
                  for (rr,cc) in cell_fringe(cell) 
                      if 0 <= r <= rr < max_iso_cell_r + 1
                      and 0 <= c-1 <= cc < max_iso_cell_c 
                      and is_on(fewer_positions, rr + rows*cc)}
    
    #if max_iso_cell_r+1 == rows-1 and is_on(fewer_positions, max_iso_cell_r+1 + rows*(max_iso_cell_c-1)):
    #    fringe_iso.add((max_iso_cell_r+1, max_iso_cell_c-1),)
    
    if False:
        for rr, cc in iso:
            if cc == size-1:
                if rr-1 >= 0 and is_on(fewer_positions, rr-1 + rows*cc):
                    fringe_iso.add((rr-1, cc),)
                if rr+1 < rows and is_on(fewer_positions, rr+1 + rows*cc):
                    fringe_iso.add((rr+1, cc),)
    
    #print("standard fringe_iso ", fringe_iso)
    
    hole = set()
    for rr, cc in fringe_iso:
        back_cc = cc
        while back_cc >= c and is_on(fewer_positions, rr + rows*back_cc):
            hole.add((rr, back_cc),)
            back_cc -= 1
            
    if hole:
        
        #print(hole)
        min_c, min_r = min((c, r) for r, c in hole)

        chances = {s for s in shapes
                   if s != shape
                   and availables[s.name]
                   and all(is_on(fewer_positions, rr + rows*cc)
                           for rr, cc in s.isomorphisms(min_r, min_c).pop())}
                   #and len(set(s.isomorphisms(0,0).pop())) <= len(hole)}

        return not chances
    else:
        return False
    
    
    
    if not fringe_iso: return False # `iso` placement fills a hole exactly

    min_c, min_r = min((c, r) for r, c in fringe_iso)
    min_cell = min_r, min_c
    #print(min_cell)
    fringe_iso = {(rr,cc) for (rr,cc) in fringe_iso if rr != min_r} | {min_cell} # discard any cell occurring more than one on the row where the min cell lies
    
    #print('doubles-free fringe ', fringe_iso)
    
    class unbounded_area(Exception): pass
    
    def ant(cell, remaining_steps):
        
        r, c = cell
        free = is_on(fewer_positions, r + rows*c)
        
        if not remaining_steps:
            return set()
            return {cell} if free else set()
        
        covered_bottom = ant((r+1, c), remaining_steps-1) if r < rows-1 else set()
        covered_right = ant((r, c+1), remaining_steps-1) if c < rows-1 else set()
        
        return {cell} | covered_bottom | covered_right
    
    for cell in fringe | fringe_iso:
        rr, cc = cell
        
        covered = ant(cell, semiperimeter-1)
        #print("covered ", covered)
        filling_shapes = {s for s in shapes 
                          if s != shape 
                          and availables[s.name] 
                          and set(s.isomorphisms(rr, cc).pop()) <= covered}
        
        if not filling_shapes:
            #print("not filling")
            return True
        
    fringe.update(fringe_iso)
    
    #print("updated fringe ", fringe)
    
    return False
    
    
    
    
    rows=size
    
    #if r == rows-1: return True
    """
    latest_placed_greater_row, latest_placed_greater_col, = max(iso)
    if latest_placed_greater_row == rows-1: # the latest shape has its greatest cell on the last row
        for q in range(0, semiperimeter-1):
            if latest_placed_greater_col-q > 0 and is_on(fewer_positions, rows*(latest_placed_greater_col-q)-1):
                return True
    """
    
    columns_ahead = 1
    for ahead in range(p+1, rows*(c+1+columns_ahead)): # look ahead up to the end of the column where last shape has been placed
        
        if not is_on(fewer_positions, ahead): continue
                
        next_free_col, next_free_row, = divmod(ahead, rows)
        
        if next_free_row == rows-1 and next_free_col > columns_ahead:
            return True
        
        """if next_free_row == rows-1:
            for q in range(2, semiperimeter):
                if not is_on(fewer_positions, rows*(next_free_col+q)-2):
                    return True
        """
        
        if not (is_on(fewer_positions, (next_free_row+1) + rows*next_free_col) or
                is_on(fewer_positions, next_free_row + rows*(next_free_col+1))):
            return True
    
    return False


In [202]:
dim = (size, size)

def nogood_predicate(placed_shape, fewer_positions, iso, availables):
    
    if not fewer_positions:
        return False
    
    shape, p = placed_shape
    
    #print("given fringe ", fringe)
    
    rows=size
    c,r = divmod(p, rows)
    
    # if this code is eventually reached, shape can be actually placed, hence 
    # there is no sense to contradict the choice just done
    #if r == rows-1 or c == size-1:
    #    return True
        
    max_r, max_c = max(iso)    
    
    if False and max_c == size-1:
        if (any(is_on(fewer_positions, rr + rows*max_c) for rr in range(0, max_r)) and
            any(is_on(fewer_positions, rr + rows*max_c) for rr in range(max_r+1, rows))):
            return True
    # m < 4^{n-1}*3
    for i in range(size >> 2):
        ahead, fewer_positions = low_bit_and_clear(fewer_positions)
        next_free_col, next_free_row, = divmod(ahead, rows)
        
        #if next_free_row == rows-1 and not is_on(fewer_positions, (next_free_row-1) + rows*next_free_col):
        #    return True
        
        if not (is_on(fewer_positions, (next_free_row+1) + rows*next_free_col) or
                is_on(fewer_positions, next_free_row + rows*(next_free_col+1))):
            return True
    
        if not fewer_positions:
            break
    
    return False
    
    for ahead in range(rows*c, rows*(c+1)): # look ahead up to the end of the column where last shape has been placed
    #for ahead in range(p+1, max_r + 1 + rows*c): # look ahead up to the end of the column where last shape has been placed
        
        if not is_on(fewer_positions, ahead): continue
                
        next_free_col, next_free_row, = divmod(ahead, rows)
        
        """if next_free_row == rows-1:
            for q in range(2, semiperimeter):
                if not is_on(fewer_positions, rows*(next_free_col+q)-2):
                    return True
        """
        
        #if next_free_row == rows-1 and next_free_col > 0:
        #    return True
        
        if not (is_on(fewer_positions, (next_free_row+1) + rows*next_free_col) or
                is_on(fewer_positions, next_free_row + rows*(next_free_col+1))):
            return True
    
    return False


In [205]:
energy=lambda item: area_key(item, {'filled':1, 'convex':0, 'bias':0})

polys_sols = polyominoes(dim, shapes, availables="ones", forbidden=[], max_depth_reached=-1,
                         simulated_annealing={'enabled':False, 'energy':energy, 'scheduler':'cauchy'},
                         nogood=nogood_predicate)

pretty_tilings = markdown_pretty(polys_sols, dim, shapes, raw_text=False)

In [206]:
next(pretty_tilings)

Solution $s_{0}$ computed in __0:00:30.838550__ time, respect last solution elapsed time (which is 0 if the current is the very first one):
	 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
	|   |     |_    |_    |   |_ _  |
	|   |_ _ _ _|_ _ _|   |_    | | |
	|   |_ _ _  |     |_ _| |_ _| |_|
	|_ _|_ _  |_|     |   |_  | |_  |
	|       |_ _|_ _ _|_ _  | |_  |_|
	|_ _ _ _| |_  | |     |_|_| |_ _|
	| |   | |   | | |_ _  |   |_ _  |
	| |   | |_  | | |_  |_|_  |_  |_|
	| |_  |   |_|_|_ _| |_  |_ _|_  |
	| | |_|_ _|   |_  |_ _|   |   |_|
	|_|_ _ _ _|_  | |_ _ _|_ _|     |
	|_ _ _ _ _| | |     |     |_ _ _|
	| |_      | |_|_ _ _|_    |_    |
	|   |_ _ _|_ _ _|   | |_ _| |_  |
	|   |   |_ _    |   |_    |   |_|
	|_ _|_ _ _ _|_ _|_ _ _|_ _|_ _ _|

and pretty print some experiments:

In [ ]:
symbols = lower_greek_symbols() + capital_greek_symbols() + latin_symbols()
symbols_map = {s.name:symbols[i] for i,s in enumerate(shapes)}
count = 0
with open('sp_6.txt', 'a') as f:
    for s in polys_sols:
        f.write('sol index {}:\n{}\n'.format(count, ppretty(s, dim, symbols_map)))
        count += 1
count

---
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Backtracking tutorial</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="massimo.nocentini@unifi.it" property="cc:attributionName" rel="cc:attributionURL">Massimo Nocentini</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/massimo-nocentini/competitive-programming/blob/master/tutorials/backtrack.ipynb" rel="dct:source">https://github.com/massimo-nocentini/competitive-programming/blob/master/tutorials/backtrack.ipynb</a>.